In [ ]:
import torch
import numpy as np
import os, sys, json, cv2, random
from glob import glob

from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.config import get_cfg

base_path = '.'

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
register_coco_instances("my_dataset_train", {}, f'{base_path}/annotations/COCO/train.json', f'{base_path}/dataset/images/')
register_coco_instances("my_dataset_val", {}, f'{base_path}/annotations/COCO/val.json', f'{base_path}/dataset/images/')

In [ ]:
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

In [ ]:
num_train_samples=len(train_dataset_dicts)
num_val_samples=len(val_dataset_dicts)

In [ ]:
batch_size = 4
num_train_steps = num_train_samples//batch_size
num_val_steps = num_val_samples//batch_size

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.OUTPUT_DIR = './Saved Models/Detectron2_models/'
cfg.DATASETS.TRAIN = ("my_dataset_train")
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.INPUT.MIN_SIZE = 640
cfg.INPUT.MIN_SIZE_TRAIN = 640
cfg.INPUT.MIN_SIZE_TEST = 640
cfg.INPUT.MAX_SIZE = 640
cfg.INPUT.MAX_SIZE_TRAIN = 640
cfg.INPUT.MAX_SIZE_TEST = 640
cfg.DATALOADER.NUM_WORKERS = 0
cfg.SOLVER.IMS_PER_BATCH = batch_size
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 300*num_train_steps   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg)
#trainer.resume_or_load(resume=False)


In [ ]:
from detectron2.evaluation import COCOEvaluator,DatasetEvaluator
from detectron2.engine.hooks import HookBase
from detectron2.data import build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper
from detectron2.data import build_detection_train_loader
from tqdm import tqdm


import logging
logging.getLogger("detectron2").setLevel(logging.INFO)



class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        train_augmentations = [
            T.ResizeShortestEdge(short_edge_length=[640,640], max_size=640),
            T.RandomBrightness(0.4, 1.6),
            T.RandomSaturation(0.3, 1.7),
            T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
            T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        ] 
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=train_augmentations))
    

class EarlyStoppingHook(HookBase):
    def __init__(self, trainer, num_train_steps, num_val_steps, max_iter):
        self.trainer = trainer
        self._period = num_train_steps
        self.num_val_steps = num_val_steps
        self._max_iter = max_iter
        self.best_model = trainer.model
        self.last_valloss = []
        self._cfg = trainer.cfg.clone()
        self._cfg.DATASETS.TRAIN = trainer.cfg.DATASETS.VAL
        self._loader = iter(build_detection_train_loader(self._cfg))

    def _do_eval(self):
        #result = self.trainer.test(self.trainer.cfg, self.trainer.model)['bbox']['AP']
        total_loss = 0
        for _ in tqdm(range(self.num_val_steps)):
            data = next(self._loader)
            with torch.no_grad():
                loss_dict = self.trainer.model(data)
                total_loss += sum(loss_dict.values())
        total_loss = total_loss/self.num_val_steps
        total_loss = total_loss.cpu().detach().numpy()

        if not np.isnan(total_loss):  
            if self.last_valloss:
                if total_loss > max(self.last_valloss):
                    self.best_model = trainer.model 
                    print('New best model!')
            if len(self.last_valloss)>=10:
                if np.argmax(self.last_valloss)==0 and total_loss < self.last_valloss[0]:
                    raise Exception("Stopping early")
                self.last_valloss[:9] = self.last_valloss[1:]  
                self.last_valloss[-1] = total_loss
            else:
                self.last_valloss.append(total_loss)
        print(self.last_valloss)
        

    def after_step(self):
        next_iter = self.trainer.iter + 1
        if next_iter % self._period == 0 or next_iter >= self._max_iter:
            self._do_eval()

trainer = MyTrainer(cfg)
trainer.register_hooks([EarlyStoppingHook(trainer, num_train_steps=num_train_steps, num_val_steps=num_val_steps, max_iter=cfg.SOLVER.MAX_ITER)])
trainer.resume_or_load(resume=False)
device = torch.device('cuda:0')
trainer.model.to(device)
trainer.train()

In [ ]:
from detectron2.evaluation import COCOEvaluator,DatasetEvaluator
from detectron2.engine.hooks import HookBase
from detectron2.data import build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper
from detectron2.data import build_detection_train_loader


import logging
logging.getLogger("detectron2").setLevel(logging.INFO)



class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        train_augmentations = [
            T.RandomBrightness(0.4, 1.6),
            T.RandomSaturation(0.3, 1.7),
            T.ResizeShortestEdge(short_edge_length=[640,640], max_size=640),
            T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
            T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        ] 
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=train_augmentations))
    

class EarlyStoppingHook(HookBase):
    def __init__(self, trainer, eval_period, max_iter):
        self.trainer = trainer
        self._period = eval_period
        self._max_iter = max_iter
        self.best_model = trainer.model
        self.last_AP = []

    def _do_eval(self):
        result = self.trainer.test(self.trainer.cfg, self.trainer.model)['bbox']['AP']
        if not np.isnan(result):  
            if self.last_AP:
                if result > max(self.last_AP):
                    self.best_model = trainer.model 
            print('New best model!')
            if len(self.last_AP)>=10:
                if np.argmax(self.last_AP)==0 and result < self.last_AP[0]:
                    raise Exception("Stopping early")
                self.last_AP[:9] = self.last_AP[1:]  
                self.last_AP[-1] = result
            else:
                self.last_AP.append(result)
            print(self.last_AP)
        

    def after_step(self):
        next_iter = self.trainer.iter + 1
        if next_iter % self._period == 0 or next_iter >= self._max_iter:
            self._do_eval()

trainer = MyTrainer(cfg)
trainer.register_hooks([EarlyStoppingHook(trainer, eval_period=num_iter, max_iter=cfg.SOLVER.MAX_ITER)])
trainer.resume_or_load(resume=False)
device = torch.device('cuda:0')
trainer.model.to(device)
#trainer.train()

In [ ]:
list(np.clip([-1,-2,1,2,3,4,5], 0,2))

In [ ]:
sys.path.append('./src')
from bounding_box import BoundingBox
from evaluators import coco_evaluator, pascal_voc_evaluator
from utils.enumerators import BBFormat, BBType, CoordinatesType
from utils.enumerators import BBFormat, CoordinatesType,  MethodAveragePrecision
import matplotlib.pyplot as plt


In [ ]:
from tqdm import tqdm
sys.path.append('./algorithms/') 
from detectron2_detector import Detectron2_detector

fasterRCNN_detector = Detectron2_detector("./Saved Models/Detectron2_models/mode_test.pth")
longest_edge_resize = 640

with open('./annotations/COCO/val.json') as json_file:
    coco_annotations = json.load(json_file)

with open('./annotations/COCO/datasets_info.json') as json_file:
    datasets_info = json.load(json_file)

VIA_datasets = {}
for dataset_name in datasets_info['datasets']:
    with open(datasets_info['datasets'][dataset_name]) as json_file:
        data = json.load(json_file)['_via_img_metadata']
        VIA_datasets[dataset_name] = {value['filename']:value for _, value in data.items()}


ann_index = 0
image_counter = 0
detected_bbs = []
groundtruth_bbs = []

for image_annotation in tqdm(coco_annotations['images']):
    id = image_annotation['id']
    file_name = image_annotation['file_name']
    true_boxes = []
    true_classes = []
    true_polygons = []
    remove_this_image = False
    while coco_annotations['annotations'][ann_index]['image_id'] == id:
        true_boxes.append(np.array(coco_annotations['annotations'][ann_index]['bbox']))
        true_classes.append(coco_annotations['annotations'][ann_index]['category_id']-1)
        true_polygons.append(np.array(coco_annotations['annotations'][ann_index]['segmentation']).reshape(4,2))
        
        ann_index+=1
        if ann_index >= len(coco_annotations['annotations']):
            break

    
    img_path = datasets_info['images'][file_name]['path']
    img = cv2.imread(img_path)
    image_counter+=1
    H,W,_ = img.shape
    if W > H:
        W_new = longest_edge_resize
        H_new = int(np.round((H*W_new)/W))
    else:
        H_new = longest_edge_resize
        W_new = int(np.round((W*H_new)/H))



    for i in range(len(true_boxes)):
        true_boxes[i][0::2] = np.int32(np.round(W_new*true_boxes[i][0::2]/W))
        true_boxes[i][1::2] = np.int32(np.round(H_new*true_boxes[i][1::2]/H))
        true_polygons[i][:,0] = np.int32(np.round(W_new*true_polygons[i][:,0]/W))
        true_polygons[i][:,1] = np.int32(np.round(H_new*true_polygons[i][:,1]/H))

    img = cv2.resize(img, (W_new, H_new), cv2.INTER_CUBIC)
    dataset_name = datasets_info['images'][file_name]['dataset']
    boxes, classes, confidences =fasterRCNN_detector.detect(img)
    detected_bbs.extend([BoundingBox(file_name, 0 if classes[i] == '1D' else 1, boxes[i], img_size=(W_new, H_new), confidence=confidences[i], bb_type=BBType.DETECTED) for i in range(len(boxes))])
    groundtruth_bbs.extend([BoundingBox(file_name, true_classes[i], true_boxes[i], img_size=(W_new, H_new), confidence=1, bb_type=BBType.GROUND_TRUTH) for i in range(len(true_boxes))])
print(coco_evaluator.get_coco_summary(groundtruth_bbs, detected_bbs))

In [ ]:
coco_evaluator.get_coco_summary(groundtruth_bbs, detected_bbs)

In [ ]:
class_id = 0
result = coco_evaluator.get_coco_summary(list(filter(lambda x: x.get_class_id() == class_id, groundtruth_bbs)),
                                list(filter(lambda x: x.get_class_id() == class_id, detected_bbs)))

Ap0 = result['AP']
class_id = 1
result = coco_evaluator.get_coco_summary(list(filter(lambda x: x.get_class_id() == class_id, groundtruth_bbs)),
                                list(filter(lambda x: x.get_class_id() == class_id, detected_bbs)))

Ap1 = result['AP']
print(Ap0, Ap1, (Ap0+Ap1)/2)

In [ ]:
x = coco_evaluator.get_coco_metrics(groundtruth_bbs, detected_bbs)[0]['interpolated recall']
y = coco_evaluator.get_coco_metrics(groundtruth_bbs, detected_bbs)[0]['interpolated precision']
plt.plot(x,y)

In [ ]:
coco_evaluator.get_coco_summary(groundtruth_bbs, detected_bbs)

In [ ]:
pascal_voc_evaluator.get_pascalvoc_metrics(groundtruth_bbs,
                          detected_bbs,
                          iou_threshold=0.5,
                          method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION,
                          generate_table=False)

In [ ]:
torch.save(trainer.model, "./Saved Models/Detectron2_models/model_final.pt")

In [ ]:
model = torch.load("./Saved Models/Detectron2_models/model_final.pt")
model.eval()

In [ ]:
#cfg = get_cfg()
cfg.MODEL.WEIGHTS = "./Saved Models/Detectron2_models/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  
cfg.DATASETS.TEST = ("my_dataset_val")
predictor = DefaultPredictor(cfg)
model = build_model(cfg)

In [ ]:
img_paths = glob('./dataset/images/*.jpg')
longest_edge_resize = 640

In [ ]:
img = cv2.imread(img_paths[0])
H,W,_ = img.shape
if W > H:
    W_new = longest_edge_resize
    H_new = int(np.round((H*W_new)/W))
else:
    H_new = longest_edge_resize
    W_new = int(np.round((W*H_new)/H))

img = cv2.resize(img, (W_new, H_new), cv2.INTER_CUBIC)

input = [{'image':torch.from_numpy(np.transpose(img, (2, 0, 1))), 
         'height':H_new, 
         'width':W_new}]

result = model(input)

In [ ]:
result_dictionary = result[0]['instances'].get_fields()

In [ ]:
result_dictionary['pred_boxes'].tensor.cpu().detach().numpy()

In [ ]:
result_dictionary['scores'].cpu().detach().numpy()

In [ ]:
result_dictionary['pred_classes'].cpu().detach().numpy()


In [ ]:
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt
import random
import time
counter = 0
predictor = DefaultPredictor(trainer)

for d in val_dataset_dicts:   
    if np.random.uniform(0,1) > 0.03:
        continue 
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=val_metadata)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[::2, ::2, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()
    counter+=1
    if counter> 100:
        break

In [ ]:
im = cv2.imread(d["file_name"])
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], metadata=val_metadata)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[::2, ::2, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
instances = outputs["instances"]
detected_class_indexes = instances.pred_classes
prediction_boxes = instances.pred_boxes

metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
#class_catalog = metadata.thing_classes

#for idx, coordinates in enumerate(prediction_boxes):
#    class_index = detected_class_indexes[idx]
#    class_name = class_catalog[class_index]
#    print(class_name, coordinates)

print(prediction_boxes, detected_class_indexes, metadata)